# **EP2 MAC0508 - Processamento de Linguagem**


## Modernização da face em Português do Corpus Paralelo através do GPT 4o mini

In [ ]:
import os
import re
import json
import time
import csv
import difflib
from typing import List, Dict, Any, Tuple
from openai import OpenAI
from google.colab import drive



# =========================
# CONFIGURAÇÕES
# =========================
INPUT_CSV = "/content/drive/MyDrive/0_tupi/tupiantigo_portugues_limpo.csv"
OUTPUT_CSV = "/content/drive/MyDrive/0_tupi/tupiantigo_portugues_modernizado.csv"

COLUNA_TEXTO = "source_text"
COLUNA_ID = "id"
COLUNA_SAIDA = "modern_source_text"

BATCH_SIZE = 30
MODEL = "gpt-4o-mini"
TEMPERATURA = 0.0

LIMIAR_SIMILARIDADE_FALLBACK = 0.3

# Relatório
SIMILARIDADE_MUITO_ALTERADA = 0.80
MAX_EXEMPLOS_IGUAIS = 15
MAX_EXEMPLOS_MUITO_ALTERADAS = 15

# Rate limit / robustez
SLEEP_ENTRE_BATCHES = 0.25
MAX_RETRIES = 6

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# =========================
# PROMPTS
# =========================
PROMPT_SISTEMA = """
Você é um assistente especializado em normalizar português antigo para português brasileiro contemporâneo.

Instruções obrigatórias:
- Reescreva cada texto preservando totalmente o sentido original.
- Não resuma, não interprete, não invente informações novas, nem substitua trechos por expressões vagas (“de verdade”, “tipo assim”, etc.).
- Não omita partes do texto: todo conteúdo informativo do original deve aparecer na versão modernizada.
- Não altere nomes próprios, datas, referências históricas ou fatos.
- Não altere capitalização (maiúsculas/minúsculas) do texto original.
- Não introduza novas frases, exemplos ou explicações.
- Não altere o tempo verbal dos verbos.
- Reescreva com fluência natural em português brasileiro contemporâneo, podendo ajustar a ordem das palavras e pontuação quando necessário, sem acrescentar conteúdo.
- Normalize apenas construções arcaicas, ortografia e conectores.
- Padronize tratamento do interlocutor para PT-BR contemporâneo:
  "a ti"→"a você"; formas de "vós" (ex.: "fazei", "dizei") → "vocês" + imperativo correspondente ("façam", "digam").
  Em ambiguidade de número, prefira "vocês" (plural).
- Em caso de dúvida sobre o significado de uma expressão arcaica, mantenha a expressão (corrigindo só ortografia/pontuação), em vez de substituir por paráfrase livre.
- Não modifique o campo "id" de cada item.
- Retorne apenas um array JSON válido, sem texto adicional, sem comentários e sem blocos de código.

Formato de saída obrigatório:
[
  {{"id": <mesmo id fornecido>, "modernizado": "<texto modernizado em português brasileiro contemporâneo>"}},
  ...
]

Requisitos estritos:
- A saída deve ser SOMENTE JSON.
- Não inclua delimitadores como ```json.
- Não mude a ordem dos itens.
- Garanta que todo texto modernizado esteja corretamente escapado em JSON.
""".strip()

PROMPT_USUARIO_TEMPLATE = """
Aqui está um lote de textos. Para cada item fornecido, retorne exatamente um objeto JSON no formato:
{{"id": <id>, "modernizado": "<texto modernizado>"}}

Regras:
- Use o mesmo "id" recebido.
- O campo "modernizado" deve conter apenas a versão modernizada do texto correspondente.
- Não repita o texto original no JSON.
- Não adicione campos extras.
- Não adicione explicações.

Lote:
{}
""".strip()

In [ ]:
# =========================
# FUNÇÕES UTILITÁRIAS
# =========================
def criar_client() -> OpenAI:
    api_key = "API_KEY" #aqui entra a chave

    return OpenAI(api_key=api_key)



_JSON_FENCE_RE = re.compile(r"^\s*```(?:json)?\s*|\s*```\s*$", re.IGNORECASE)
def extrair_json_puro(conteudo: str) -> str:
    s = (conteudo or "").strip()
    s = _JSON_FENCE_RE.sub("", s).strip()

    start = s.find("[")
    end = s.rfind("]")
    if start != -1 and end != -1 and end > start:
        s = s[start:end + 1].strip()

    return s


def similaridade(a: str, b: str) -> float:
    return difflib.SequenceMatcher(None, a, b).ratio()


def garantir_colunas(linhas: List[Dict[str, Any]]) -> Tuple[List[str], List[Dict[str, Any]]]:
    if not linhas:
        raise RuntimeError("CSV vazio.")
    if COLUNA_TEXTO not in linhas[0]:
        raise RuntimeError(f"Coluna '{COLUNA_TEXTO}' não existe no CSV.")


    if COLUNA_ID not in linhas[0]:
        for i, row in enumerate(linhas):
            row[COLUNA_ID] = str(i)
        print(f"[Info] Coluna '{COLUNA_ID}' não existia; criei IDs sequenciais.")
    else:
        for row in linhas:
            row[COLUNA_ID] = str(row.get(COLUNA_ID, "")).strip()


    for row in linhas:
        if COLUNA_SAIDA not in row:
            row[COLUNA_SAIDA] = ""

    fieldnames = list(linhas[0].keys())
    if COLUNA_SAIDA not in fieldnames:
        fieldnames.append(COLUNA_SAIDA)

    return fieldnames, linhas


def carregar_csv(path: str) -> List[Dict[str, Any]]:
    with open(path, "r", encoding="utf-8", newline="") as f:
        return list(csv.DictReader(f))


def salvar_csv(path: str, linhas: List[Dict[str, Any]], fieldnames: List[str]) -> None:
    with open(path, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(linhas)



In [ ]:
# =========================
# API
# =========================
def modernizar_batch(client: OpenAI, batch: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    lote_json = json.dumps(batch, ensure_ascii=False)
    prompt_usuario = PROMPT_USUARIO_TEMPLATE.format(lote_json)

    last_err = None
    for attempt in range(MAX_RETRIES):
        try:
            resp = client.chat.completions.create(
                model=MODEL,
                temperature=TEMPERATURA,
                messages=[
                    {"role": "system", "content": PROMPT_SISTEMA},
                    {"role": "user", "content": prompt_usuario},
                ],
            )
            conteudo = (resp.choices[0].message.content or "").strip()
            json_puro = extrair_json_puro(conteudo)
            data = json.loads(json_puro)

            if not isinstance(data, list):
                raise ValueError("JSON retornado não é uma lista.")

            for item in data:
                if not isinstance(item, dict) or "id" not in item or "modernizado" not in item:
                    raise ValueError(f"Item inválido no JSON: {item}")

            return data

        except Exception as e:
            last_err = e
            delay = min(2 ** attempt, 20) + (0.05 * attempt)
            print(f"[Aviso] Erro no batch (tentativa {attempt+1}/{MAX_RETRIES}): {e}")
            print(f"[Aviso] Aguardando {delay:.2f}s e tentando novamente...")
            time.sleep(delay)

    raise RuntimeError(f"Falha após {MAX_RETRIES} tentativas. Último erro: {last_err}")


In [ ]:
# =========================
# BATCHING / APLICAÇÃO
# =========================
def construir_batch(linhas: List[Dict[str, Any]], start: int, end: int) -> Tuple[List[Dict[str, Any]], Dict[str, int]]:
    fatia = linhas[start:end]
    batch = []
    index_por_id = {}

    for offset, row in enumerate(fatia):
        abs_idx = start + offset
        texto = str(row.get(COLUNA_TEXTO, "") or "")
        rid = str(row.get(COLUNA_ID, "")).strip()

        if rid == "":
            rid = str(abs_idx)
            row[COLUNA_ID] = rid

        if texto.strip() == "":
            linhas[abs_idx][COLUNA_SAIDA] = ""
            continue

        batch.append({"id": rid, "texto": texto})
        index_por_id[rid] = abs_idx

    return batch, index_por_id


def aplicar_resultados(
    linhas: List[Dict[str, Any]],
    resultados: List[Dict[str, Any]],
    index_por_id: Dict[str, int],
    iguais: List[Tuple[str, str]],
    muito_alteradas: List[Tuple[str, str, str, float]],
) -> None:
    for item in resultados:
        rid = str(item["id"]).strip()
        modern = str(item["modernizado"])

        if rid not in index_por_id:
            continue

        abs_idx = index_por_id[rid]
        orig = str(linhas[abs_idx].get(COLUNA_TEXTO, "") or "")

        o = orig.strip()
        m = modern.strip()

        s = similaridade(o, m) if (o and m) else 1.0

        #Limiar de Fallback
        if s < 0.30:
            # grava o original no CSV
            linhas[abs_idx][COLUNA_SAIDA] = orig
            continue
        else:
            # grava a versão modernizada
            linhas[abs_idx][COLUNA_SAIDA] = modern

        # Relatório
        if o == m:
            if len(iguais) < MAX_EXEMPLOS_IGUAIS:
                iguais.append((rid, o))
        else:
            if s < SIMILARIDADE_MUITO_ALTERADA and len(muito_alteradas) < MAX_EXEMPLOS_MUITO_ALTERADAS:
                muito_alteradas.append((rid, o, m, s))


def imprimir_relatorio(n: int, iguais, muito_alteradas, output_path: str) -> None:
    print("\n================ RELATÓRIO ================\n")
    print(f"Arquivo de saída: {output_path}")
    print(f"Linhas totais: {n}")

    print(f"\nExemplos 'iguais' (até {MAX_EXEMPLOS_IGUAIS}): {len(iguais)}")
    for rid, o in iguais:
        print(f"- ID {rid}: {o}")

    print(
        f"\nExemplos 'muito alteradas' (similaridade < {SIMILARIDADE_MUITO_ALTERADA}, "
        f"até {MAX_EXEMPLOS_MUITO_ALTERADAS}): {len(muito_alteradas)}"
    )
    for rid, o, m, s in muito_alteradas:
        print(f"\n- ID {rid} | similaridade={s:.3f}")
        print(f"  Original:    {o}")
        print(f"  Modernizado: {m}")

    print("\nProcesso concluído.")

In [ ]:
# =========================
# PIPELINES
# =========================

#Função para modernizar todo o corpus
def processar_csv() -> None:
    client = criar_client()

    linhas = carregar_csv(INPUT_CSV)
    fieldnames, linhas = garantir_colunas(linhas)

    n = len(linhas)
    total_batches = (n + BATCH_SIZE - 1) // BATCH_SIZE

    iguais: List[Tuple[str, str]] = []
    muito_alteradas: List[Tuple[str, str, str, float]] = []

    for b in range(total_batches):
        start = b * BATCH_SIZE
        end = min(start + BATCH_SIZE, n)

        batch, index_por_id = construir_batch(linhas, start, end)

        if not batch:
            print(f"[Batch {b+1}/{total_batches}] vazio (só textos vazios). Pulando.")
            continue

        print(f"[Batch {b+1}/{total_batches}] Processando {len(batch)} itens...")
        resultados = modernizar_batch(client, batch)

        aplicar_resultados(linhas, resultados, index_por_id, iguais, muito_alteradas)

        salvar_csv(OUTPUT_CSV, linhas, fieldnames)  # incremental
        time.sleep(SLEEP_ENTRE_BATCHES)

    imprimir_relatorio(n, iguais, muito_alteradas, OUTPUT_CSV)


#Função que moderniza uma amostra das N primeiras sentenças (utilizado no ajuste do prompt)
def processar_csv_amostra(sample_size: int = 10) -> None:
    client = criar_client()

    linhas = carregar_csv(INPUT_CSV)
    _, linhas = garantir_colunas(linhas)

    batch = []
    for row in linhas:
        if len(batch) >= sample_size:
            break
        texto = str(row.get(COLUNA_TEXTO, "") or "")
        if texto.strip():
            batch.append({"id": str(row[COLUNA_ID]), "texto": texto})

    if not batch:
        print("Nada para processar (todos vazios).")
        return

    resultados = modernizar_batch(client, batch)
    mapa = {str(x["id"]): str(x["modernizado"]) for x in resultados}

    print("\n===== AMOSTRA =====\n")
    for item in batch:
        rid = str(item["id"])
        orig = item["texto"].strip()
        modern = mapa.get(rid, "").strip()


        # só printa as frases alteradas para análise das alterações
        if not (orig == modern):
            print(f"ID {rid}")
            print(f"Original:    {orig}")
            print(f"Modernizado: {modern}")
            s = similaridade(orig, modern)
            print(f"Status: similaridade={s:.3f}\n")

In [ ]:
# Para rodar o processamento completo:
processar_csv()

[Info] Coluna 'id' não existia; criei IDs sequenciais.
[Batch 1/232] Processando 30 itens...
[Batch 2/232] Processando 30 itens...
[Batch 3/232] Processando 30 itens...
[Batch 4/232] Processando 30 itens...
[Batch 5/232] Processando 30 itens...
[Batch 6/232] Processando 30 itens...
[Batch 7/232] Processando 30 itens...
[Batch 8/232] Processando 30 itens...
[Batch 9/232] Processando 30 itens...
[Batch 10/232] Processando 30 itens...
[Batch 11/232] Processando 30 itens...
[Batch 12/232] Processando 30 itens...
[Batch 13/232] Processando 30 itens...
[Batch 14/232] Processando 30 itens...
[Batch 15/232] Processando 30 itens...
[Batch 16/232] Processando 30 itens...
[Batch 17/232] Processando 30 itens...
[Batch 18/232] Processando 30 itens...
[Batch 19/232] Processando 30 itens...
[Batch 20/232] Processando 30 itens...
[Batch 21/232] Processando 30 itens...
[Batch 22/232] Processando 30 itens...
[Batch 23/232] Processando 30 itens...
[Batch 24/232] Processando 30 itens...
[Batch 25/232] Pro

Amostra com o prompt atual

In [ ]:
# Para rodar o teste rápido:
processar_csv_amostra(sample_size=100)

[Info] Coluna 'id' não existia; criei IDs sequenciais.

===== AMOSTRA =====

ID 0
Original:    aparei as pontas deles
Modernizado: aparar as pontas deles
Status: similaridade=0.909

ID 10
Original:    tu te comprazes quando um homem apalpa teus seios?
Modernizado: você se compraz quando um homem apalpa seus seios?
Status: similaridade=0.880

ID 12
Original:    afastaste teu filho e teu escravo de suas amantes?
Modernizado: afastou seu filho e seu escravo de suas amantes?
Status: similaridade=0.898

ID 15
Original:    matai-o!
Modernizado: matem-no!
Status: similaridade=0.706

ID 22
Original:    estando doente, de verdade, não a ouviste, ou sendo preguiçoso?
Modernizado: estando doente, de verdade, não a ouviu, ou sendo preguiçoso?
Status: similaridade=0.968

ID 25
Original:    que se alegre esta vossa terra
Modernizado: que se alegre esta sua terra
Status: similaridade=0.931

ID 26
Original:    confesso-me a ti, senhor padre, por causa do meu pecar muitas vezes
Modernizado: confesso-me